# CNN Facial Recognition

## Introduction
Hey guys, the Stats Whisper is back with another task at hand. This time we will be using a convulution neural network to resolve a doopleganger debate. 





## CNN Explianed

- images
- videos 
- mathematically explained


# Application

# 1. Creating Basic CNN using Local Data

In [23]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [24]:
batch_size = 5 

train_datagen = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(250, 250),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='binary')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)



Found 70 images belonging to 2 classes.
Found 70 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10,
        callbacks = [tensorboard])
model.save_weights('first_try.h5')  # always save your weights after training or during training

# 2. Using VGGFace on Local Data

In [1]:
from keras.engine import  Model
from keras_vggface.vggface import VGGFace
from keras.models import Sequential,Input,Model
from keras import layers
from keras.layers import Dense, Flatten,Input,Dropout,Activation,Conv2D
import keras

#custom parameters
nb_class = 2
hidden_dim = 512

base_model = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='max')
last_layer = base_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
#x = Dense(hidden_dim, activation='relu')(x)
x=Dropout(0.5)(x)
x = Dense(hidden_dim, activation='relu')(x)
out = Dense(nb_class, activation='softmax')(x)
custom_vgg_model = Model(base_model.input, out)


batch_size = 64
epochs = 20

custom_vgg_model.compile(loss=keras.losses.categorical_crossentropy,
         optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')
from time import time
batch_size = 5 

train_datagen = ImageDataGenerator(data_format="channels_last")

test_datagen = ImageDataGenerator(data_format="channels_last")

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 250x250
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

tensorboard = TensorBoard(log_dir="tensorBoard/".format(time()), write_images=True, write_graph=True)

custom_vgg_model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=10)


Found 70 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Epoch 1/5
16/20 [=======================>......] - ETA: 30s - loss: 10.2351 - acc: 0.3500

KeyboardInterrupt: 